In [1]:
# Chargement des packages
import random
import re
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree
from scipy.stats import norm
import functools
import time

# Fonction qui permet de récuperer le code html d'un page internet
# ATTENTION : IL FAUT MODIFIER L'USER AGENT !!!!!!

# Décorateur pour rajouter un cache à la fonction get_page.
# Ceci est fait pour gagner du temps et ne pas avoir à charger plusieurs fois les pages quand on utilise plusieurs fois la fonction finale
@functools.cache
def get_page(urlpage): 
    """
    Récupération du HTML d'un site internet via Beautifulsoup
    """
    user_agent = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0'}
    time.sleep(0.2 + np.random.rand()/10) # Retarder le téléchargement pour pas se faire ban
    res = requests.get(urlpage, headers = user_agent)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def get_links_page_lefigaro(urlpage):
    """
    Stockage des url des différentes pages du sites "lefigaro" dans un vecteur où sont stockés les annonces
    
    Tant que la page possède un bouton suivant à la fin de la liste des annonces, on cherche l'url de la page suivante et on la stocke dans le vecteur `pages`
    """
    website = "https://immobilier.lefigaro.fr" 
    link = None
    pages = [urlpage]
    while link != "VIDE":
        soup = get_page(urlpage) 
        text = soup.find_all("a", "router-link-active router-link-exact-active btn-pagination") # Récupération du html de la page suivante
        for i in range(len(text)): # Il y a 2 élements dans "text" : "Précédent" et "Suivant".
            if text[i].text == "Suivant": # On cherche l'élement "Suivant"
                link = text[i].get("href") # Et on cherche le lien qui est après "href="
                urlpage = website + link # On merge le nom du site et la partie du lien spécifique à la page pour avoir le lien complet
                pages.append(urlpage) # On ajoute le lien au vecteur "page" pour le stocker
                break
            else:
                link = "VIDE"
    return pages

def get_link(site):
    """
    Récupération des liens html de toutes les annonces d'un site
    
    site : str  
        valeur : "orpi", "nexity" ou "lefigaro"
    """
    links = []
    # Récupération des liens pour 'orpi'
    if (site=='orpi')==True:
        soup = get_page('https://www.orpi.com/annonces-immobilieres-strasbourg/vente-appartement/')
        annonces = soup.find_all("a", class_= 'u-link-unstyled c-overlay__link')
        for annonce in annonces:
            link = annonce.get("href")
            path = "https://www.orpi.com" + link 
            links.append(path)
        
    # Récupération des liens pour 'nexity'
    if (site=='nexity')==True:
        soup = get_page('https://www.nexity.fr/annonces-immobilieres/achat-vente/appartement/tout/strasbourg+67')
        annonces = soup.find_all("h2", class_= 'informations')
        for annonce in annonces:
            link = annonce.a.get("href")
            if "neuf" in link: # Exclusion des "neuf" car il s'agit de projet de construction
                continue
            path = "https://www.nexity.fr" + link # 
            links.append(path)
    
    # Récupération des liens pour 'lefigaro'
    if (site=='lefigaro')==True:
        
        # Comme il y a plusieurs pages sur le site "lefigaro", il faut prendre le lien des annonces pour toutes les pages
        page_link = get_links_page_lefigaro("https://immobilier.lefigaro.fr/annonces/immobilier-vente-appartement-strasbourg+67000.html")# On récupère les liens des différentes pages de lefigaro

        for page in page_link:
            soup = get_page(page)
            annonces = soup.find_all("a", class_= 'content__link')
            for annonce in annonces: # Pour chaque annonce, on récupère le lien html
                link = annonce.get("href")
                if "https" not in link: # exclusion des autres sites proposés par lefigaro
                    path = "https://immobilier.lefigaro.fr" + link # On regroupe le lien du site et le lien de l'annonce pour avoir le lien complet
                    links.append(path) # On l'ajoute à notre vecteur "links"
        
    return np.unique(links)

# Orpi

In [2]:
# a) Localisation : Ville et Quartier
def localisation_orpi(soup):
    """
    Récupération de la ville et du quartier pour un logement d'une annonce du site "orpi".
    """
    ville = soup.find("span", class_='u-h3 u-ml-xs u-text-normal').text
    quartiers = ["meinau", "neustadt", 'poteries', "esplanade", "petite france", "cronenbourg", "koenigshoffen", "halles", "neudorf", "robertsau", "gare", "musau", "orangerie", "krutenau", "forêt noire", 'centre-ville', "port du rhin", "hautepierre", "contades", "tribunal", "neuhof", "montagne verte"]
    text = soup.find_all("h2", class_='u-h3')
    if ville == "Strasbourg":
        for i in range(len(text)):
            if "Quartier" in text[i].text:
                quartier = re.findall(r'Quartier (.*?) à', text[i].text)
                for j in range(len(quartiers)):
                    if quartier[0].lower().find(quartiers[j]) != -1:
                        quartier = quartiers[j].capitalize()
    else:
        quartier = ville
    return [ville, quartier]

# b) Prix et Honoraires
def price_orpi(soup): 
    """
    Récupération du prix et du montant des honoraires pour un logement d'une annonce "orpi".
    """
    prix = float(soup.find("span", class_='u-h1 u-color-primary').text.replace('\xa0', '').replace('€', ''))
    text = soup.find("ul", class_='u-list-unstyled u-text-xs u-mt-xs u-color-text-grey').find_all('li')
    honoraires = float('nan')
    for i in range(len(text)):
        element = text[i].text
        if "Honoraire" in element:
            if "acquéreur" in element:
                honoraires = prix * float(re.search(r'\b\d+(?:\.\d+)?', element).group(0))/100
            else:
                honoraires = 0
    return [prix, honoraires]

# c) Pièces et Surface
def feature_orpi(soup):
    """
    Récupération de la surface (nombre de m²) et du nombre de pièce d'un logement d'une annonce "orpi".
    """
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    piece = float('nan')
    surface = float('nan')
    for i in range(len(text)):
        if "pièce" in text[i].find("span").text:
            piece = float(re.findall(r'\d+', text[i].find("span").text)[0])
        if "Surface" in text[i].find("span").text:
            surface = float(re.search(r'\d+[\.,]?\d*', text[i].find("span").text).group(0).replace(',', '.'))
    if piece == 'nan' or surface == 'nan': # Si piece et surface sont toujours "nan", les caractéristiques sont peut être à un autre endroit
        text = soup.find_all(class_='u-block@sm u-block@md-plus')[1].text.split()
        for i in range(len(text)):
            if piece == 'nan' and "pièce" in text[i]:
                piece = float(text[i-1])
            if surface == 'nan' and "m2" in text[i]:
                surface = float(text[i-1])
    return [piece, surface]

# d) Caractéristiques
def options_orpi(soup):
    """
    Récupération des options disponibles pour un logement d'une annonce "orpi".
    Options : "Ascenseur", "Balcon", "Terrasse", "Cave", "Garage", "Terrain".
    """
    variables = ["ascenseur", "balcon", "terrasse", "cave", "garage", "terrain"]
    text = soup.find_all(class_='u-flex u-flex-cross-center')
    caracteristiques = list(np.zeros(len(variables), dtype=float))
    for i in range(len(text)-1):
        for j in range(len(variables)):
            if variables[j] in text[i].find("span").text.lower():
                caracteristiques[j] = float(1)
    return caracteristiques

# Fonction qui nous donne toutes les caractéristiques d'une annonce orpi à l'aide des fonctions précédentes 
def get_orpi(urlpage):
    """
    Extraction des caractéristiques d'un logement d'une annonce du site "orpi" et stockage dans un vecteur.
    
    urlpage : str
        Url d'une annonce "orpi"
    """
    soup = get_page(urlpage)
    appart = localisation_orpi(soup) + price_orpi(soup) + feature_orpi(soup) + options_orpi(soup) + [urlpage]
    return appart

# Nexity

In [3]:
# a) Localisation : Ville et Quartier
def localisation_nexity(soup):
    """
    Récupération de la ville et du quartier pour un logement d'une annonce du site "nexity".
    """
    ville = soup.find("span", class_='city').text
    if ville == "Strasbourg":
        quartiers = ["meinau", "neustadt", 'poteries', "esplanade", "petite france", "cronenbourg", "koenigshoffen", "halles", "neudorf", "robertsau", "gare", "musau", "orangerie", "krutenau", "forêt noire", 'centre-ville', "port du rhin", "hautepierre", "contades", "tribunal", "neuhof", "montagne verte"]
        text = soup.find("div", class_='description text_body_1 mt-2').text.lower().split()
        quartier = "nan"
        for i in range(len(quartiers)):
            for j in range(len(text)):
                if quartiers[i] in text[j]:
                    quartier = quartiers[i].capitalize()
                    break
    else:
        quartier = ville
    return [ville, quartier]

# b) Prix et honoraires :
def price_nexity(soup):
    """
    Récupération du prix et du montant des honoraires pour un logement d'une annonce "nexity".
    """
    prix = float(soup.find_all("h2", class_="text text--small text--hightlighted text--hightlighted--not-new")[0].text.replace(" €", "").replace(" ", ""))
    text = soup.find("div", class_="text text--small--honoraires").text
    if "vendeur" in text:
        honoraires = float(0)
    else:
        honoraires = float("nan")
    return [prix, honoraires] 

# c) caractéristiques : "pièce(s)", "surface", "ascenseur", "balcon", "terrasse", "cave", "parking" et "terrain"
def options_nexity(soup):  
    """
    Récupération des options disponibles pour un logement d'une annonce "nexity".
    Options : "Pièces", "surface", "Ascenseur", "Balcon", "Terrasse", "Cave", "Garage", "Terrain".
    """
    variables = ["pièce(s)", "surface", "ascenseur", "balcon", "terrasse", "cave", "parking", "terrain"]
    caracteristiques = list(np.zeros(len(variables), dtype=float))
    text = soup.find_all("div", class_='d-flex align-items-center')
    for i in range(len(text)):
        element = text[i].text.lower()
        for j in range(len(variables)):
            if variables[j] in element:
                caracteristiques[j] = element.split()[len(element.split())-1].capitalize()
                break
    caracteristiques[1] = caracteristiques[1].replace('m²', '')
    for i in [0,1]:
        caracteristiques[i] = float(caracteristiques[i])
    for j in [2,3,4,5,6,7]:
        if caracteristiques[j] == "Non":
            caracteristiques[j] = float(0)
        else:
            caracteristiques[j] = float(1)
    return caracteristiques

# Fonction qui nous donne toutes les caractéristiques d'une annonce nexity à l'aide des fonction précédente 
def get_nexity(urlpage):
    """
    Extraction des caractéristiques d'un logement d'une annonce du site "nexity" et stockage dans un vecteur.
    
    urlpage : str
        Url d'une annonce "nexity"
    """
    soup = get_page(urlpage)
    appart = options_nexity(soup) + [urlpage]
    appart.insert(0, localisation_nexity(soup)[0])
    appart.insert(1, localisation_nexity(soup)[1])
    appart.insert(2, price_nexity(soup)[0])
    appart.insert(3, price_nexity(soup)[1])
    return appart

# Lefigaro

In [4]:
# a) localisation : ville et quartier
def localisation_lefigaro(soup):
    """
    Récupération de la ville et du quartier pour un logement d'une annonce du site "lefigaro".
    """
    ville = re.search(r'à (\w+)', soup.find_all('h1')[0].text.lower()).group(1).capitalize()
    quartiers = ["meinau", "neustadt", 'poteries', "esplanade", "petite france", "cronenbourg", "koenigshoffen", "halles", "neudorf", "robertsau", "gare", "musau", "orangerie", "krutenau", "forêt noire", 'centre-ville', "port du rhin", "hautepierre", "contades", "tribunal", "neuhof", "montagne verte"]
    text = soup.find_all("p", class_='truncated-description')[0].text.lower()
    quartier = "nan"
    if ville=="Strasbourg":
        for i in range(len(quartiers)):
            if text.find(quartiers[i]) != -1:
                quartier = quartiers[i].capitalize()
    else:    
        quartier = ville
    return [ville, quartier]

# b) Loyer et charges
def price_lefigaro(soup):
    """
    Récupération du prix et du montant des honoraires pour un logement d'une annonce "lefigaro".
    """
    price = float(soup.find_all('div', class_='classified-price-per-m2')[0].find("strong").text.replace(" €", "").replace(" ", ""))
    text = soup.find_all('li', class_='item-about-price')
    honoraires = float("nan")
    for i in range(len(text)):
        if "honoraires" in text[i].text.lower():
            if "vendeur" in text[i].text.lower():
                honoraires = float(0)
                break
            elif "non communiqué" in text[i].text.lower():
                honoraires = float("nan")
                break
            else:
                honoraires = price * float(re.search(r'\b\d+(?:\.\d+)?', text[i].text.lower()).group(0))/100
                break
    return [price, honoraires]

# c) Pièces, surface
def feature_lefigaro(soup):
    """
    Récupération de la surface (nombre de m²) et du nombre de pièce d'un logement d'une annonce "lefigaro".
    """
    piece = float("nan")
    surface = float("nan")
    text= soup.find_all('span', class_='feature')
    for i in range(len(text)):
        element = text[i].text.split()
        if "pièces" in element or 'pièce' in element:
            piece = float(re.match(r'([\d.]+)', element[0]).group())
        if "surface" in element:
            surface = float(re.match(r'([\d.]+)', element[0]).group())
    return [piece, surface]

# d) Caractéristiques : Ascenseur, "balcon", "terrasse", "cave", "parking" et "jardin"
def options_lefigaro(soup):
    """
    Récupération des options disponibles pour un logement d'une annonce "lefigaro".
    Options : "Ascenseur", "Balcon", "Terrasse", "Cave", "Garage", "Terrain".
    """
    variables = ["ascenseur", "balcon", "terrasse", "cave", "parking", "jardin"]
    caracteristiques = list(np.zeros(len(variables), dtype=float))
    text = soup.find_all("li", class_='options')
    if text != []:
        for i in range(len(text)):
            for j in range(len(variables)):
                if variables[j] == text[i].text.lower():
                    caracteristiques[j] = float(1)
    return caracteristiques

def get_lefigaro(urlpage):
    """
    Extraction des caractéristiques d'un logement d'une annonce du site "lefigaro" et stockage dans un vecteur.
    
    urlpage : str
        Url d'une annonce "lefigaro"
    """
    soup = get_page(urlpage)
    appart = localisation_lefigaro(soup) + price_lefigaro(soup) + feature_lefigaro(soup) + options_lefigaro(soup) + [urlpage]
    return appart

# Data

In [5]:
def get_appart(site):
    """
    Création d'un data frame qui contient les caractéristiques des annonces d'un site immobilier spécifique.
    
    site : str
        Valeur : "orpi", "nexity" ou "lefigaro"
    """
    links = get_link(site)
    var = ["Ville", "Quartier", "Prix", "Honoraires", "Pièces", "Surface", "Ascenseur", "Balcon", "Terrasse", "Cave", "Parking", "Terrain", "Liens"]
    data = []
    z=0 # Initialisation d'un compteur
    if (site=="orpi")==True:
        for i in range(len(links)):
            z+=1
            print("Site : Orpi - Chargement : ", round((z/len(links))*100), "%", end="\r") # Affichage du compteur pour indiquer l'évolution du chargement des données
            info = get_orpi(links[i]) # On récupère le vecteur des caractéristiques d'un logement à partir des liens que nous avons récupérés
            data.append(info) # On ajoute notre observation au vecteur data qui regroupe toutes les observations
    if (site=="nexity")==True:
        for i in range(len(links)):
            z+=1
            print("Site : Nexity - Chargement : ", round((z/len(links))*100), "%", end="\r")
            info = get_nexity(links[i])
            data.append(info)
    if (site=="lefigaro")==True:
        for i in range(len(links)):
            z+=1
            print("Site : Lefigaro - Chargement : ", round((z/len(links))*100), "%", end="\r")
            info = get_lefigaro(links[i])
            data.append(info)
    data = pd.DataFrame(data, columns=var)
    return data

def data_immo(site):
    """
    Création d'un data frame qui contient les caractéristiques de toutes les annonces pour les sites "orpi", "nexity" et "lefigaro" avec traitement des valeurs manquantes et des doublons.
    
    site : str
        Valeur : "orpi", "nexity" ou "lefigaro"
    """
    t = time.time()
    print("Extraction et chargement des données...\n")
    if site != "all":
        dt = get_appart(site)
    else:
        dt = pd.concat([get_appart("orpi"), get_appart("nexity"), get_appart("lefigaro")], ignore_index=True)
    
    # Traitement des valeurs manquantes
    dt = dt.dropna(subset=["Prix", "Surface", 'Pièces', 'Terrain', 'Ascenseur', 'Balcon', 'Terrasse','Cave','Parking'])
    dt = dt.reset_index(drop=True)
    
    #Traitement des valeurs doubles
    doublons = []
    for i in range(len(dt)):
        if dt[["Prix", "Pièces", "Surface"]].duplicated()[i] == True:
            doublons.append(i)
    for j in range(len(doublons)):
        dt = dt.drop(doublons[j])
    dt = dt.reset_index(drop=True)
    print("Extraction terminée en ", round((time.time() - t)/60, 2), "minutes")
    return dt

In [6]:
dt = data_immo("all")
dt

Extraction et chargement des données...

Extraction terminée en  8.82 minutes%


,Ville,Quartier,Prix,Honoraires,Pièces,Surface,Ascenseur,Balcon,Terrasse,Cave,Parking,Terrain,Liens
0,Strasbourg,Krutenau,209000.0,6186.4,1.0,40.00,0.0,0.0,0.0,1.0,0.0,0.0,https://www.orpi.com/annonce-vente-appartement...
1,Strasbourg,Centre-ville,114500.0,0.0,1.0,17.59,0.0,0.0,0.0,0.0,0.0,0.0,https://www.orpi.com/annonce-vente-appartement...
2,Strasbourg,Krutenau,169000.0,6219.2,1.0,29.47,0.0,0.0,0.0,0.0,0.0,0.0,https://www.orpi.com/annonce-vente-appartement...
3,Strasbourg,Krutenau,126000.0,5997.6,1.0,20.88,0.0,0.0,0.0,0.0,0.0,0.0,https://www.orpi.com/annonce-vente-appartement...
4,Strasbourg,Krutenau,139000.0,6268.9,1.0,29.31,0.0,0.0,0.0,0.0,0.0,0.0,https://www.orpi.com/annonce-vente-appartement...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,Strasbourg,Centre-ville,473000.0,0.0,4.0,81.00,1.0,0.0,0.0,0.0,1.0,1.0,https://immobilier.lefigaro.fr/annonces/annonc...
399,Strasbourg,nan,225480.0,0.0,3.0,66.00,1.0,1.0,0.0,0.0,1.0,0.0,https://immobilier.lefigaro.fr/annonces/annonc...
400,Strasbourg,nan,368480.0,0.0,4.0,85.00,1.0,1.0,0.0,0.0,0.0,0.0,https://immobilier.lefigaro.fr/annonces/annonc...
401,Strasbourg,nan,384460.0,0.0,4.0,95.00,1.0,1.0,0.0,0.0,0.0,0.0,https://immobilier.lefigaro.fr/annonces/annonc...


In [11]:
def annonces():
    """
    Fonctions qui permet d'obtenir une liste d'annonces selon des caractéristiques spécifiées.
    """
    # Demandes informations
    Information ={"Quel est le prix que vous visez ?" : "",
                  "Quelle surface souhaitez-vous ? (en m²)" : "",  
                  "Combien de pièces souhiatez-vous ?" : "",        
                  "Souhaitez-vous du terrain ?" : "",
                  "Souhaitez-vous un ascenseur ?" : "", 
                  "Souhaitez-vous un balcon ?" : "",   
                  "Souhaitez-vous une terrasse ?" : "", 
                  "Souhaitez-vous une cave ?" : "",   
                  "Souhaitez-vous une place de statonnement (parking ou garage) ?" : "",
                  }
    
    key = list(Information.keys())
    print("Demande d'informations :\n")
    
    # Affichage de la demande d'information
    for i in range(len(Information)):
        print(key[i], ":")
        element = input().lower()
        if "prix" in key[i] or "surface" in key[i] or "pièces" in key[i]:
            while element.isnumeric() == False:
                if element.isnumeric() == False:
                    print("Erreur : Veuillez insérer une valeur numérique")
                    element = input()
            element = float(element)
        else:
            while element != "oui" and element != "non":
                if element != "oui" and element != "non":
                    print("Erreur : Veuillez indiquer si vous souhaitez cette caractéristique ou non\nRéponses acceptées : Oui ou Non")
                    element = input().lower()
            element = element.capitalize()
        Information[list(Information.keys())[i]]=element
    
    # importation des données des logements
    #dt = import_data("all")    
    a=dt[["Prix", "Surface", 'Pièces', 'Terrain', 'Ascenseur', 'Balcon', 'Terrasse','Cave','Parking']]
    b=dt["Liens"]
    
    # Transformation des variables d'inforamtion de text en boolean et stockage dans X
    X = []
    for i in range(len(key)):
        if (Information.get(key[i])=="Oui")==True:
            info = 1
        elif (Information.get(key[i])=="Non")==True:
            info = 0
        else :
            info = Information.get(key[i])
        X.append(info)
    X = pd.DataFrame([X], columns=a.columns)
    
    # Entrainement du modèle knn
    model = KNeighborsClassifier(n_neighbors=4)
    model.fit(a,b)
    distances, indices = model.kneighbors(X)
    
    # Affichage des résultats de la fonction
    data = dt.iloc[indices[0]][["Liens", "Prix", "Surface", "Pièces", "Quartier", "Honoraires"]]
    for i in range(len(data)):
        if data.iloc[i][4]=="nan":
            quart = "Non renseigné"
        else:
            quart = data.iloc[i][4]
        print("\nAnnonce",i+1,":")
        print("Quartier : ", quart)
        print("Prix : ", int(data.iloc[i][1]), "€\n" "Honoraires : ", data.iloc[i][5], "€\n" "Surface : ", data.iloc[i][2], "m2","\n" "Pièces : ", int(data.iloc[i][3]),"\n" "Lien : ", data.iloc[i][0])

In [8]:
def estimation():
    """
    Fonctions qui permet d'obtenir une estimation d'un bien immobilier selon des caractéristiques spécifiées.
    """
    # Demandes informations
    Information = {"Quelle est la surface en m² de votre bien" : "",  
                  "Nombre de pièces" : "",        
                  "Avez-vous du terrain ?" : "",
                  "Avez-vous un ascenseur ?" : "", 
                  "Avez-vous un balcon ?" : "",   
                  "Avez-vous une terrasse ?" : "", 
                  "Avez-vous une cave ?" : "",   
                  "Avez-vous une place de statonnement (parking ou garage) ?" : "",
                  }
    
    key = list(Information.keys())
    print("Demande d'informations :\n")

    # Affichage de la demande d'information
    for i in range(len(Information)):
        print(key[i], ":")
        element = input().lower()
        if "prix" in key[i] or "surface" in key[i] or "pièces" in key[i]:
            while element.isnumeric() == False:
                if element.isnumeric() == False:
                    print("Erreur : Veuillez insérer une valeur numérique")
                    element = input()
            element = float(element)
        else:
            while element != "oui" and element != "non":
                if element != "oui" and element != "non":
                    print("Erreur : Veuillez indiquer si vous souhaitez cette caractéristique ou non\nRéponses acceptées : Oui ou Non")
                    element = input().lower()
            element = element.capitalize()
        Information[list(Information.keys())[i]]=element

    # importation des données des logements
    #dt = import_data("all")    
    a=dt[["Surface", 'Pièces', 'Terrain', 'Ascenseur', 'Balcon', 'Terrasse', 'Cave', 'Parking']]
    b=dt["Prix"]
    
    # Transformation des variables d'inforamtion de text en boolean et stockage dans X
    X = []
    for i in range(len(key)):
        if (Information.get(key[i])=="Oui")==True:
            ad = 1
        elif (Information.get(key[i])=="Non")==True:
            ad = 0
        else :
            ad = Information.get(key[i])
        X.append(ad)
    X = pd.DataFrame([X], columns=a.columns)
    
    # Entrainement du modèle knn
    model = KNeighborsClassifier(n_neighbors=4)
    model.fit(a,b)
    distances, indices = model.kneighbors(X)
    
    # Estimation du prix
    prix = []
    for i in range(len(indices[0])):    
        obs = dt.iloc[indices[0][i]]["Prix"]
        prix.append(obs)
    prix = np.array(prix)
    z = distances.sum()- distances[0]
    sum_z = z.sum()
    pond = z/sum_z
    estimation = (prix*pond).sum()
    
    # Intervalle de confiance
    lower_bound = (loy*pond).sum() -  norm.ppf((1.95) / 2) * (np.std(loy) / np.sqrt(len(loy)))
    upper_bound = (loy*pond).sum() + norm.ppf((1.95) / 2) * (np.std(loy) / np.sqrt(len(loy)))
    
    # Affichage des résultats de la fonction
    print("")
    print("Estimation du prix : ", "{:,}".format(estimation.round()).replace(","," "), "€")
    print("Interval de confiance du prix : [","{:,}".format(lower_bound.round()).replace(","," "), "€", ";","{:,}".format(upper_bound.round()).replace(","," "), "€", "]")

In [9]:
estimation()

Demande d'informations :

Quelle est la surface en m² de votre bien :


 60


Nombre de pièces :


 3


Avez-vous du terrain ? :


...

Avez-vous une cave ? :


 oui


Avez-vous une place de statonnement (parking ou garage) ? :


 oui



Estimation du prix :  246 330.0 €
Interval de confiance du prix : [ 167 161.0 € ; 325 499.0 € ]


In [13]:
annonces()

Demande d'informations :

Quel est le prix que vous visez ? :


 100000


Quelle surface souhaitez-vous ? (en m²) :


 20


Combien de pièces souhiatez-vous ? :


...

Souhaitez-vous une cave ? :


 non


Souhaitez-vous une place de statonnement (parking ou garage) ? :


 non



Annonce 1 :
Quartier :  Neudorf
Prix :  100000 €
Honoraires :  9890.0 €
Surface :  24.0 m2 
Pièces :  1 
Lien :  https://www.orpi.com/annonce-vente-appartement-t1-strasbourg-67100-3690-054021-216/

Annonce 2 :
Quartier :  Neudorf
Prix :  99950 €
Honoraires :  4167.915 €
Surface :  24.25 m2 
Pièces :  1 
Lien :  https://immobilier.lefigaro.fr/annonces/annonce-66564626.html

Annonce 3 :
Quartier :  Contades
Prix :  99800 €
Honoraires :  0.0 €
Surface :  20.09 m2 
Pièces :  1 
Lien :  https://www.orpi.com/annonce-vente-appartement-t1-strasbourg-67000-6564-054006-285/

Annonce 4 :
Quartier :  Non renseigné
Prix :  96550 €
Honoraires :  0.0 €
Surface :  43.0 m2 
Pièces :  2 
Lien :  https://www.nexity.fr/ancien/GB00179111
